# DF + LIB

In [1]:
# df that have:
# user as numeric feature (from 0 to 5) 
%store -r merge_unique_numain_df

In [2]:
# package that we need
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.metrics import MeanAbsolutePercentageError
from tensorflow.keras.regularizers import l2

#for the CROSS-VALIDATION
from sklearn.model_selection import KFold

# essential
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# not necessary in this case
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,accuracy_score, precision_score, recall_score,f1_score

# CROSS VALIDATION 

we want to try this approach in order to optimize our recomandation system, more in detail our NN

# GENARALIZE CRSS VALIDATION - all users

In [3]:
# TARGET_var
target='valence'

#FEATURES
features=[    'spotify_track_duration_minute', 'danceability', 'energy', 'key', 'loudness',
    'mode', 'speechiness', 'acousticness', 'tempo', 'time_signature',
    'spotify_track_popularity', 'instrumentalness', 'liveness']

## DEFINE FUNCTIONS

In [4]:
# function to create the model
def create_model_user(shape_x_train): #number of columns -> shape
    
    
    '''model_user_i = Sequential([
        Dense(32, input_dim=shape_x_train, activation='relu', name='first_layer'),
        # 64 as the number of initial layer is a common choice when we build a NN --> VEDIAMO COME FUNZIA POI NEL CASO SI SISTEMA IN SEGUITO
        # input_dim: take the dimension of the x_train array --> that is equivalent to the number of feature in input
        
        # SI POSSONO AGGIUNGERE PER OGNI LAYER --> Dropout(0.3) per evitare overfitting
        

        Dense(16, activation='relu', name='secod_layer'),
        Dense(1, activation='linear', name='final_layer')
    ])'''

    model_user_i = Sequential([
        
        Dense(32,input_dim=shape_x_train,activation='relu',name='first_layer',kernel_regularizer=l2(0.01)),
        # 64 as the number of initial layer is a common choice when we build a NN --> VEDIAMO COME FUNZIA POI NEL CASO SI SISTEMA IN SEGUITO
        # input_dim: take the dimension of the x_train array --> that is equivalent to the number of feature in input
        
        # SI POSSONO AGGIUNGERE PER OGNI LAYER --> Dropout(0.3) per evitare overfitting
        Dropout(0.3),
        Dense(16,activation='relu',name='secod_layer',kernel_regularizer=l2(0.01)),
        Dropout(0.2),
        #Dense(1,activation='linear',name='final_layer')
        Dense(1,activation='sigmoid',name='final_layer') 
        #ATENZION PROVIAMO SIGMOID PRCHè CI SERVE UN OUTPUT NEL RANGE DI O E 1
    
    ])

    #compile the model
    model_user_i.compile(optimizer = 'adam', loss = 'mse', metrics = ['mae'])

    #summary
    #display(model_user_i.summary())

    #return  the model
    return model_user_i

#function to train the model
def fit_model(model, x_train, y_train, x_test, y_test):

    history_model = model.fit(
        x_train,y_train,
        #validation_split=0.2,
        validation_data = [x_test, y_test],
        epochs = 1,
        #epochs=100,
        batch_size=32,
        #verbose=1
    )

    return history_model

def plot_for_model(history, y_hat, x_test):

    #plot loss
    plt.figure(figsize=(10,6))
    plt.plot(history.history['loss'],color='blue',label='loss')
    plt.plot(history.history['val_loss'],color='red',label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss function through epoch')
    plt.legend()
    plt.grid()
    plt.show()

    #plot mean
    plt.figure(figsize=(10,6))
    plt.plot(history.history['mae'],color='blue',label='mae')
    plt.plot(history.history['val_mae'],color='red',label='val_mae')
    plt.xlabel('Epoch')
    plt.ylabel('mae')
    plt.title('Mae function through epoch')
    plt.grid()
    plt.legend()
    plt.show()   

def return_train_test_for_cross(train_index, test_index, df):
    #extract the feature and target
    x = df[features]
    y = df[target]

    #normalization
    scaler = StandardScaler()
    x[features] = scaler.fit_transform(x[features])
    #display(x)

    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    #display(x_train)

    return x_train,x_test,y_train,y_test

def reco_function_conf_matrix(y_test, y_hat, threshold):
    
    #initialize dataframe    
    data = {'Original Test': y_test,
        'Prediction Test': y_hat.flatten(),
        }
    df_prediction = pd.DataFrame(data = data)
    df_prediction['Original over Thrashold'] = df_prediction['Original Test'] > threshold
    df_prediction['Predicted over Thrashold'] = df_prediction['Prediction Test'] > threshold
    #display(df_prediction)

    #plot confusion matrix of reccomandation
    cm = confusion_matrix(df_prediction['Original over Thrashold'], df_prediction['Predicted over Thrashold'])

    # METRIX FOR EVALUATE RECCOMANDATION 
    # Extract TP, TN, FP, FN
    TN, FP, FN, TP = cm.ravel()

    # Calculate metrics manually
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    '''plt.figure()
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['True', 'False'], yticklabels=['True', 'False'])
    plt.title('Confiusion matrix')
    plt.xlabel('predicted')
    plt.ylabel('actual')
    plt.show()'''

    # Store metrics in a DataFrame
    df_metrics_cm = pd.DataFrame(
        data =
        #'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-score'],
        {'Value': [accuracy, precision, recall, f1]},
        index = ['Accuracy', 'Precision', 'Recall', 'F1-score']
    )

    #display(df_metrics_cm.round(2))

    return df_prediction,df_metrics_cm

## MAIN

In [5]:
dizio_model_user = {} #in this dizio we store the model for each user
dizio_accuracy = {}

# DICTIONARY TO STORE SONG RECOMMENDATION PER USER
reccomandations_songs_per_user={}

#threshold
threshold = 0.7

#value of folder
k = 5

#loop creation user and model
for i in range(0, len(merge_unique_numain_df['User'].unique())): #i indica lo user   
    
    print('---------------------------------------------------------')
    print(f'USER {i}')
    
    #def dizio
    df_metrics_kfold = pd.DataFrame()

    #select user
    mask_user = merge_unique_numain_df['User'] == i
    #display(mask_user)
    user_i_df = merge_unique_numain_df[mask_user]
    #display(user_i_df) 

    kf = KFold(n_splits = k, shuffle= True, random_state= 42)
    score_fold = []

    loop = 0
    for train_index, test_index in kf.split(user_i_df):

        print('---------------------------------------------------------')
        print(f'K {loop} FOLD') #print(f'K {} FOLD = test set')
        loop = loop + 1

        #return train and test
        x_train,x_test,y_train,y_test = return_train_test_for_cross(train_index, test_index, user_i_df) #da modificare

        #creation model for the specific user
        model_user_i = create_model_user(x_train.shape[1])
        
        #fit the model with the different x, y train (for different user)
        history_model_user_i = fit_model(model_user_i, x_train, y_train, x_test, y_test)

        y_hat = model_user_i.predict(x_test)

        test_loss, test_mae = model_user_i.evaluate(x_test,y_test,verbose=1)
        score_fold.append(test_mae)

        #reccomandation
        reccomandation_df = reco_function_conf_matrix(y_test, y_hat, threshold)
        df_metrics_kfold = pd.concat([df_metrics_kfold,reccomandation_df[1]], axis = 1)
        
    dizio_model_user[f"User{i}"] = {#'model_user_i' : model_user_i,
                                    #'history_model_user_i': history_model_user_i,
                                    #'x_train': x_train,
                                    #'x_test': x_test,
                                    #'y_train': y_train,
                                    #'y_test': y_test,
                                    #'y_hat': y_hat,
                                    #'recommendations:': reccomandation_df
                                    'score_fold': score_fold,
                                    'df_metrics_kfold_mean': pd.DataFrame(data = df_metrics_kfold.mean(axis=1), columns=['Mean_Metrics'])
                                    } 
    #save accuracy 
    dizio_accuracy[f'User{i}'] = {
        #'MAE' : mean_absolute_error(y_test, y_hat),
        #'MSE' : mean_squared_error(y_test, y_hat)
        'Mean MAE': np.mean(score_fold)
    }

---------------------------------------------------------
USER 0
---------------------------------------------------------
K 0 FOLD


C:\Users\elped\AppData\Local\Temp\ipykernel_14044\4280919785.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[features] = scaler.fit_transform(x[features])
c:\Users\elped\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


143/143 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.3764 - mae: 0.2012 - val_loss: 0.1769 - val_mae: 0.1667
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1764 - mae: 0.1660 
---------------------------------------------------------
K 1 FOLD


C:\Users\elped\AppData\Local\Temp\ipykernel_14044\4280919785.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[features] = scaler.fit_transform(x[features])
c:\Users\elped\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


143/143 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.3751 - mae: 0.2235 - val_loss: 0.1747 - val_mae: 0.1796
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1712 - mae: 0.1714 
---------------------------------------------------------
K 2 FOLD


...

63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.4318 - mae: 0.2388 - val_loss: 0.3006 - val_mae: 0.1799
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2975 - mae: 0.1722 
---------------------------------------------------------
K 3 FOLD


C:\Users\elped\AppData\Local\Temp\ipykernel_14044\4280919785.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[features] = scaler.fit_transform(x[features])
c:\Users\elped\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.4528 - mae: 0.2336 - val_loss: 0.3192 - val_mae: 0.1813
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3172 - mae: 0.1768 
---------------------------------------------------------
K 4 FOLD


C:\Users\elped\AppData\Local\Temp\ipykernel_14044\4280919785.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[features] = scaler.fit_transform(x[features])
c:\Users\elped\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.4599 - mae: 0.2313 - val_loss: 0.3281 - val_mae: 0.1983
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3265 - mae: 0.1934 


# DISPLAY DICTIONARY ACCURAY

In [6]:
display(dizio_accuracy)

{'User0': {'Mean MAE': 0.17344275414943694},
 'User1': {'Mean MAE': 0.1949895739555359},
 'User2': {'Mean MAE': 0.1265860229730606},
 'User3': {'Mean MAE': 0.18013178408145905},
 'User4': {'Mean MAE': 0.19088200628757476}}

# DISPLAY DF METRICS CONFUSION MATRIX

In [7]:
dizio_model_user['User4']['df_metrics_kfold_mean']

,Mean_Metrics
Accuracy,0.634417
Precision,0.546667
Recall,0.022554
F1-score,0.042743


In [8]:
df_metrics_total = pd.concat([dizio_model_user['User0']['df_metrics_kfold_mean'],dizio_model_user['User1']['df_metrics_kfold_mean'],dizio_model_user['User2']['df_metrics_kfold_mean'],dizio_model_user['User3']['df_metrics_kfold_mean'],dizio_model_user['User4']['df_metrics_kfold_mean']], axis=1)
#type(df_metrics_total)
#df_metrics_total.columns(colume_name = ['User0','User1','User2','User3','User4'])
df_metrics_total.columns = ['User0','User1','User2','User3','User4']
display(df_metrics_total)

,User0,User1,User2,User3,User4
Accuracy,0.611815,0.630758,0.699912,0.759806,0.634417
Precision,0.404113,0.725002,0.820381,0.166667,0.546667
Recall,0.013005,0.058931,0.126588,0.002511,0.022554
F1-score,0.025116,0.107945,0.210017,0.004945,0.042743


In [10]:
df_metrics_mean = pd.DataFrame(data = df_metrics_total.mean(axis=1), columns=['Mean_Metrics'])
display(df_metrics_mean.round(3))

,Mean_Metrics
Accuracy,0.667
Precision,0.533
Recall,0.045
F1-score,0.078
